In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import joblib
from sklearn.inspection import partial_dependence, plot_partial_dependence
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
sys.path.append('../src')
from preprocess import HTRUPreprocessor
from utils import VisualizationUtils

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 300

print("HTRU2 Pulsar Detection - Model Interpretability")
print("=" * 50)

# Load best model and data
best_model = joblib.load("../models/RandomForest_best.pkl")['model']  # Update if different best model
scaler = joblib.load("../models/scaler.pkl")

# Initialize preprocessor to get feature names
preprocessor = HTRUPreprocessor()
_, _, X_test, _, _, y_test = preprocessor.prepare_data()
feature_names = preprocessor.feature_names

C:\Users\tahak\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'plot_partial_dependence' from 'sklearn.inspection' (C:\Users\tahak\AppData\Roaming\Python\Python312\site-packages\sklearn\inspection\__init__.py)

In [ ]:
print("\nSHAP Value Analysis")
print("=" * 50)

# Initialize SHAP explainer
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test)

# Summary plot
plt.figure()
shap.summary_plot(shap_values[1], X_test, feature_names=feature_names, show=False)
plt.title("SHAP Feature Importance")
plt.tight_layout()
plt.savefig("../results/figures/shap_summary.png", dpi=300)
plt.show()

# Force plot for single observation
print("\nSHAP Force Plot for Single Prediction (Pulsar):")
sample_idx = np.where(y_test == 1)[0][0]  # First pulsar in test set
shap.force_plot(explainer.expected_value[1], shap_values[1][sample_idx,:], 
                X_test[sample_idx,:], feature_names=feature_names, matplotlib=True)
plt.title(f"SHAP Explanation for Pulsar Sample (True Class)")
plt.tight_layout()
plt.savefig("../results/figures/shap_force_pulsar.png", dpi=300)
plt.show()

In [ ]:
print("\nPartial Dependence Analysis")
print("=" * 50)

# Get top 3 most important features
importances = best_model.feature_importances_
top_features = np.argsort(importances)[-3:][::-1]

# Create PDP plots
fig, ax = plt.subplots(1, 3, figsize=(18, 5))
for i, feature_idx in enumerate(top_features):
    plot_partial_dependence(
        best_model, X_test, [feature_idx], 
        feature_names=feature_names,
        ax=ax[i]
    )
    ax[i].set_title(f"PDP for {feature_names[feature_idx]}")
plt.tight_layout()
plt.savefig("../results/figures/partial_dependence.png", dpi=300)
plt.show()

In [ ]:
print("\nError Analysis")
print("=" * 50)

# Get predictions
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:,1]

# Identify misclassified samples
fp_mask = (y_test == 0) & (y_pred == 1)  # False positives
fn_mask = (y_test == 1) & (y_pred == 0)  # False negatives

# Compare feature distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# False Positives vs True Negatives
for i, feature in enumerate(feature_names[:4]):
    sns.kdeplot(X_test[y_test==0, i], label="True Negatives", ax=axes[0,0])
    sns.kdeplot(X_test[fp_mask, i], label="False Positives", ax=axes[0,0])
axes[0,0].set_title("Feature Distributions: FP vs TN")
axes[0,0].legend()

# False Negatives vs True Positives
for i, feature in enumerate(feature_names[4:8]):
    sns.kdeplot(X_test[y_test==1, 4+i], label="True Positives", ax=axes[0,1])
    sns.kdeplot(X_test[fn_mask, 4+i], label="False Negatives", ax=axes[0,1])
axes[0,1].set_title("Feature Distributions: FN vs TP")
axes[0,1].legend()

# Confidence scores
sns.histplot(y_proba[y_test==0], label="Non-Pulsars", kde=True, ax=axes[1,0])
sns.histplot(y_proba[y_test==1], label="Pulsars", kde=True, ax=axes[1,0])
axes[1,0].axvline(0.5, color='red', linestyle='--')
axes[1,0].set_title("Prediction Confidence Distribution")
axes[1,0].legend()

# Error cases by feature importance
error_features = X_test[fp_mask | fn_mask][:, top_features]
sns.boxplot(data=error_features, ax=axes[1,1])
axes[1,1].set_xticks(range(len(top_features)))
axes[1,1].set_xticklabels([feature_names[i] for i in top_features], rotation=45)
axes[1,1].set_title("Top Features in Misclassified Samples")

plt.tight_layout()
plt.savefig("../results/figures/error_analysis.png", dpi=300)
plt.show()

In [ ]:
print("\nDecision Threshold Optimization")
print("=" * 50)

from sklearn.metrics import precision_recall_curve

# Calculate precision-recall tradeoff
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba)

# Find optimal threshold (maximizing F1)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]

plt.figure()
plt.plot(thresholds, precisions[:-1], label="Precision")
plt.plot(thresholds, recalls[:-1], label="Recall")
plt.plot(thresholds, f1_scores[:-1], label="F1")
plt.axvline(optimal_threshold, color='red', linestyle='--')
plt.xlabel("Decision Threshold")
plt.ylabel("Score")
plt.title("Precision-Recall Tradeoff")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("../results/figures/threshold_optimization.png", dpi=300)
plt.show()

print(f"\nOptimal Decision Threshold: {optimal_threshold:.3f}")
print(f"At this threshold:")
print(f"- Precision: {precisions[optimal_idx]:.3f}")
print(f"- Recall: {recalls[optimal_idx]:.3f}")
print(f"- F1 Score: {f1_scores[optimal_idx]:.3f}")

In [ ]:
print("\n" + "="*50)
print("INTERPRETABILITY SUMMARY")
print("="*50)

print(f"""
1. Key Findings:
   - Most important features: {feature_names[top_features[0]]}, {feature_names[top_features[1]]}
   - False positives tend to have higher values in profile statistics
   - False negatives often have DM-SNR curve features similar to non-pulsars

2. Model Behavior:
   - Higher prediction confidence for true pulsars
   - Decision boundary could be optimized (current threshold: 0.5)
   - Suggested optimal threshold: {optimal_threshold:.2f}

3. Recommendations:
   - Consider feature engineering for problematic features
   - Adjust decision threshold based on use case (precision vs recall)
   - Monitor model performance on new data with similar characteristics
""")